In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
# Function to calculate RSI
def calculate_rsi(df, column='Close', period=14):
    delta = df[column].diff()
    up_changes = delta.where(delta > 0, 0)
    down_changes = -delta.where(delta < 0, 0)

    if len(df) < period:
        # Not enough data toac calculate RSI
        return pd.Series([np.nan] * len(df), index=df.index)

    # Calculate initial averages of gains and losses
    initial_avg_up = up_changes.rolling(window=period).mean().dropna().iloc[0]
    initial_avg_down = down_changes.rolling(window=period).mean().dropna().iloc[0]

    # Lists to hold subsequent smoothed averages
    avg_up = [initial_avg_up]
    avg_down = [initial_avg_down]

    # Calculate subsequent averages using Wilder's smoothing method
    for i in range(period, len(up_changes)):
        avg_up.append((avg_up[-1] * (period - 1) + up_changes.iloc[i]) / period)
        avg_down.append((avg_down[-1] * (period - 1) + down_changes.iloc[i]) / period)

    avg_up_series = pd.Series(avg_up, index=df.index[period - 1:])
    avg_down_series = pd.Series(avg_down, index=df.index[period - 1:])

    # Calculate RSI
    rs = avg_up_series / avg_down_series
    rsi = 100 - (100 / (1 + rs))

    # Prepend NaNs for the initial period where RSI is not available
    rsi = pd.concat([pd.Series([np.nan] * (period - 1), index=df.index[:period - 1]), rsi])

    return rsi

In [3]:
# Function to get historical data for a ticker
start = '2023-01-01'
end = '2024-01-01'  # This can be None if you want data up to the current date

def get_ticker_data(tickerSymbol, start, end=None, interval='1d'):
    tickerData = yf.Ticker(tickerSymbol)
    # Fetch historical data within the specified date range
    df = tickerData.history(start=start, end=end, interval=interval, auto_adjust=False)

    from talib import RSI
    df['RSI'] = RSI(df['Close'], timeperiod=14)
    df['RSI'] = df['RSI'].shift(1)

    from talib import MA, SMA, EMA, WMA
    df['MA'] = MA(df['Close'], timeperiod=9)
    df['MA'] = df['MA'].shift(1)
    df['SMA'] = SMA(df['Close'], timeperiod=9)
    df['SMA'] = df['SMA'].shift(1)
    df['WMA'] = WMA(df['Close'], timeperiod=9)
    df['WMA'] = df['WMA'].shift(1)

    from talib import MACD
    df['macd'], df['signal'], df['hist'] = MACD(df['Close'])
    df['macd'] = df['macd'].shift(1)
    df['signal'] = df['signal'].shift(1)
    df['hist'] = df['hist'].shift(1)

    from talib import ADX
    df['adx'] = ADX(df['High'], df['Low'], df['Close'])
    df['adx'] = df['adx'].shift(1)
    
    from talib import ATR
    df['atr'] = ATR(high=df['High'], low=df['Low'], close=df['Close'], timeperiod=14)
    df['atr'] = df['atr'].shift(1)
    
    from talib import SAR
    df['sar'] = SAR(high=df['High'], low=df['Low'], acceleration=0.02, maximum=0.2)
    df['sar'] = df['sar'].shift(1)
    
    from talib import TEMA
    df['tema'] = TEMA(df['Close'], timeperiod=14)
    df['tema'] = df['tema'].shift(1)
    
    from talib import ROC
    df['roc'] = ROC(df['Close'], timeperiod=14)
    df['roc'] = df['roc'].shift(1)
    
    df['Date'] = pd.to_datetime(df.index)
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['EOM'] = df['Date'] == df.groupby(['Year', 'Month'])['Date'].transform('max')
    # No change to the data processing code that follows

    df.dropna(axis=0, inplace=True)

#   # Define the path for the data directory
#    data_directory = "/content/data"
#    # Create the directory if it does not exist
#    if not os.path.exists(data_directory):
#        os.makedirs(data_directory)

#    # Define the full path for the CSV file within the data directory
#    raw_data_filename = os.path.join(data_directory, f'{tickerSymbol}_raw_data.csv')
#    # Save the DataFrame to CSV in the specified directory
#    df.to_csv(raw_data_filename, index=True)  # Include the datetime index in the CSV

#    print(f"Raw data for {tickerSymbol} saved to {raw_data_filename}")
    return df

In [4]:
df = get_ticker_data('KCHOL.IS', start, end)

In [5]:
df

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,RSI,MA,...,hist,adx,atr,sar,tema,roc,Date,Month,Year,EOM
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-03 00:00:00+03:00,80.150002,82.800003,77.199997,78.900002,77.165901,31451178,0.0,0.0,56.372192,76.983332,...,0.714504,18.108116,3.710712,68.871956,79.212208,5.424839,2023-03-03 00:00:00+03:00,3,2023,False
2023-03-06 00:00:00+03:00,79.400002,84.650002,78.900002,84.500000,82.642822,31118918,0.0,0.0,52.517115,77.294444,...,0.651378,16.910218,3.845662,69.425078,79.499743,14.430754,2023-03-06 00:00:00+03:00,3,2023,False
2023-03-07 00:00:00+03:00,84.800003,87.199997,84.000000,84.000000,82.153809,29398907,0.0,0.0,61.573139,78.327777,...,0.935577,16.095386,3.981686,70.227573,81.632678,28.030303,2023-03-07 00:00:00+03:00,3,2023,False
2023-03-08 00:00:00+03:00,84.099998,89.900002,83.750000,89.500000,87.532928,25534993,0.0,0.0,60.464306,79.338888,...,1.031154,15.953463,3.925851,71.381368,83.013044,10.817937,2023-03-08 00:00:00+03:00,3,2023,False
2023-03-09 00:00:00+03:00,89.500000,89.849998,87.250000,87.550003,85.625793,17048208,0.0,0.0,67.415553,80.938889,...,1.384051,16.398842,4.084719,72.963231,85.908706,19.412947,2023-03-09 00:00:00+03:00,3,2023,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25 00:00:00+03:00,140.399994,140.600006,137.000000,137.000000,137.000000,6084344,0.0,0.0,46.547464,142.655557,...,0.054417,11.709525,3.829402,138.498674,143.059410,-1.610646,2023-12-25 00:00:00+03:00,12,2023,False
2023-12-26 00:00:00+03:00,138.100006,139.199997,136.100006,137.500000,137.500000,8821245,0.0,0.0,41.443390,142.477778,...,-0.332231,11.726466,3.813017,149.000000,141.000224,-5.517241,2023-12-26 00:00:00+03:00,12,2023,False
2023-12-27 00:00:00+03:00,137.500000,138.300003,135.300003,136.000000,136.000000,9411433,0.0,0.0,42.414835,142.566667,...,-0.525972,11.968581,3.762086,148.760000,139.632231,-3.032442,2023-12-27 00:00:00+03:00,12,2023,False


In [6]:
import numpy as np

# Create input sequences and targets
def create_sequences(features, target, seq_length):
    X_seq = []
    y_seq = []
    for i in range(len(features) - seq_length):
        X_seq.append(features[i:i+seq_length])  # Input sequence
        y_seq.append(target[i+seq_length]) # Target value (next data point)
    return np.array(X_seq), np.array(y_seq)

In [7]:
from sklearn.preprocessing import MinMaxScaler

df_input = df[['Close', 'MA', 'SMA', 'macd', 'signal', 'hist', 'adx', 'atr', 'sar', 'tema', 'roc']]

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_input)

scaler_target = MinMaxScaler()
target_scaled = scaler_target.fit_transform(df[['Close']])

In [8]:
seq_length = 4  # Number of time steps in each sequence
num_features = df_input.shape[1]
X_features = df_scaled
y_target = target_scaled

In [9]:
X_b, y_b = create_sequences(df_scaled, target_scaled, seq_length)
X_b = X_b.reshape(X_b.shape[0], seq_length, num_features)

In [10]:
import pickle

# Load the pickled model from file
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

2024-04-05 07:39:15.263685: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 07:39:15.341651: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 07:39:15.341745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 07:39:15.345294: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-05 07:39:15.360152: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 07:39:17.296837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [11]:
y_pred = loaded_model.predict(X_b)
y_pred = scaler_target.inverse_transform(y_pred)

7/7 [==============================] - 1s 4ms/step


In [12]:
y_pred[2][0], len(df)

(84.124466, 209)

In [20]:
def find_manual_peaks(df, y_pred):
    peaks = []
    for i in range(seq_length, len(df)-1):
        if df['Close'].iloc[i] > df['Close'].iloc[i-1] and df['Close'].iloc[i] > y_pred[i+1-seq_length][0]:
            peaks.append(i)
    return peaks

def find_manual_troughs(df, y_pred):
    troughs = []
    for i in range(seq_length, len(df)-1):
        if df['Close'].iloc[i] < df['Close'].iloc[i-1] and df['Close'].iloc[i] < y_pred[i+1-seq_length][0]:
            troughs.append(i)
    return troughs

In [21]:
def detect_divergences(df, tolerance=0.01):
    divergences = []

    peaks_indices = find_manual_peaks(df, y_pred)
    troughs_indices = find_manual_troughs(df, y_pred)

    def process_divergence(indices, find_highs):
        if len(indices) >= 2:
            for i in range(len(indices) - 1):
                # Fetch the index, price, and RSI values for the peaks/troughs
                idx1, idx2 = indices[i], indices[i+1]
                price1, price2 = df['Close'].iloc[idx1], df['Close'].iloc[idx2]
                rsi1, rsi2 = df['RSI'].iloc[idx1], df['RSI'].iloc[idx2]

                if find_highs:
                    if price1 > price2 and rsi1 > rsi2:
                        divergences.append(('Bearish', df.index[idx1], df.index[idx2], price1, price2, rsi1, rsi2))
                else:
                    if price1 < price2 and rsi1 < rsi2:
                        divergences.append(('Bullish', df.index[idx1], df.index[idx2], price1, price2, rsi1, rsi2))

    # Process both peaks and troughs
    process_divergence(peaks_indices, True)
    process_divergence(troughs_indices, False)

    return divergences

In [22]:
def process_tickers(tickerSymbols):
    all_divergences = []
    for ticker in tickerSymbols:
        #try:
        print(f"Processing {ticker}...")
        df = get_ticker_data(ticker, start, end)
        # Shift the close prices up to get the next day's close
        df['Next Bar Close'] = df['Close'].shift(-1)

        #if len(df) >= 14:  # Ensure there's enough data to calculate RSI
        divergences = detect_divergences(df)

        # Append divergence details, ensuring correct alignment with the DataFrame columns
        for divergence in divergences:
            # Find the index for the next bar close
            idx2 = divergence[2]  # This is the index for 'Second Date'
            next_bar_close = df.at[idx2, 'Next Bar Close'] if idx2 in df.index else np.nan

            all_divergences.append(
                (
                    ticker,  # Ticker
                    divergence[0],  # Type of divergence
                    divergence[1],  # First Date
                    divergence[2],  # Second Date
                    divergence[3],  # First Price
                    divergence[4],  # Second Price
                    divergence[3],  # Use First Price again as "First Close Price"
                    divergence[4],  # Use Second Price again as "Second Close Price"
                    divergence[5],  # First RSI
                    divergence[6],  # Second RSI
                    df['Close'].iloc[-1],  # Final Close Price
                    df.index[-1],  # Final Date
                    next_bar_close   # Next Bar Close
                )
            )
        #except Exception as e:
        #    print(f"Error processing {ticker}: {e}")
    return all_divergences

In [27]:
# Path to your tickers file
#file_path = "/content/tickers.txt"
file_path = "tickers-Turkey.txt"

# Initialize an empty list to hold the ticker symbols
tickerSymbols = []

# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        # Strip newline characters and any leading/trailing whitespace
        ticker = line.strip()
        # Add the ticker symbol to the list
        tickerSymbols.append(ticker)

In [28]:
# After generating all divergences
all_divergences = process_tickers(tickerSymbols)

# Create the DataFrame
divergences_df = pd.DataFrame(all_divergences, columns=[
    'Ticker', 'Type',
    'First Date', 'Second Date',
    'First Price', 'Second Price',
    'First Close', 'Second Close',
    'First RSI', 'Second RSI',
    'Final Close', 'Final Date',
    'Next Bar Close'  # Include the new column for Next Bar Close
])

Processing Symbol...


SYMBOL: No timezone found, symbol may be delisted


Processing ISKUR.IS...
Processing QNBFB.IS...
Processing KCHOL.IS...
Processing FROTO.IS...
Processing THYAO.IS...
Processing TUPRS.IS...
Processing ISCTR.IS...
Processing ISBTR.IS...
Processing ISATR.IS...
Processing GARAN.IS...
Processing ASELS.IS...
Processing BIMAS.IS...
Processing AKBNK.IS...
Processing ENKAI.IS...
Processing SASA.IS...
Processing KENT.IS...
Processing YKBNK.IS...
Processing SAHOL.IS...
Processing QNBFL.IS...
Processing CCOLA.IS...
Processing EREGL.IS...
Processing SISE.IS...
Processing VAKBN.IS...
Processing TCELL.IS...
Processing TOASO.IS...
Processing ISDMR.IS...
Processing BRYAT.IS...
Processing TTKOM.IS...
Processing HALKB.IS...
Processing ARCLK.IS...
Processing BRSAN.IS...
Processing TTRAK.IS...
Processing AEFES.IS...
Processing MGROS.IS...
Processing OYAKC.IS...
Processing KLNMA.IS...
Processing PGSUS.IS...
Processing POLTK.IS...
Processing KOZAL.IS...
Processing ENJSA.IS...
Processing AGHOL.IS...
Processing TAVHL.IS...
Processing GUBRF.IS...
Processing PET

PATEK.IS: Data doesn't exist for startDate = 1672520400, endDate = 1704056400


Processing ALCAR.IS...
Processing VERUS.IS...
Processing TMSN.IS...
Processing EGPRO.IS...
Processing ASUZU.IS...
Processing BJKAS.IS...
Processing JANTS.IS...
Processing IZENR.IS...
Processing LMKDC.IS...


LMKDC.IS: Data doesn't exist for startDate = 1672520400, endDate = 1704056400


Processing MAGEN.IS...
Processing CANTE.IS...
Processing TKFEN.IS...
Processing BANVT.IS...
Processing SARKY.IS...
Processing BIENY.IS...
Processing YYLGD.IS...
Processing ISGYO.IS...
Processing DEVA.IS...
Processing KMPUR.IS...
Processing SRVGY.IS...
Processing TETMT.IS...
Processing PEKGY.IS...
Processing ULUSE.IS...
Processing ODAS.IS...
Processing ERCB.IS...
Processing OYYAT.IS...
Processing GWIND.IS...
Processing NATEN.IS...
Processing INGRM.IS...
Processing OZKGY.IS...
Processing MEGMT.IS...
Processing ADEL.IS...
Processing RYSAS.IS...
Processing ICBCT.IS...
Processing DGGYO.IS...
Processing KLKIM.IS...
Processing NTHOL.IS...
Processing GARFA.IS...
Processing DAPGM.IS...
Processing AKSGY.IS...
Processing VAKKO.IS...
Processing TUKAS.IS...
Processing UFUK.IS...
Processing BERA.IS...
Processing AGESA.IS...
Processing KAREL.IS...
Processing SNPAM.IS...
Processing BUCIM.IS...
Processing ATATP.IS...
Processing DOKTA.IS...
Processing RYGYO.IS...
Processing PSGYO.IS...
Processing IPEKE.

BORSK.IS: Data doesn't exist for startDate = 1672520400, endDate = 1704056400


Processing ARDYZ.IS...
Processing TKNSA.IS...
Processing PARSN.IS...
Processing FENER.IS...
Processing PKENT.IS...
Processing LOGO.IS...
Processing AKMGY.IS...
Processing SUNTK.IS...
Processing BASCM.IS...
Processing PENTA.IS...
Processing INDES.IS...
Processing SKBNK.IS...
Processing HTTBT.IS...
Processing GOKNR.IS...
Processing GLYHO.IS...
Processing KERVT.IS...
Processing PRKAB.IS...
Processing BSOKE.IS...
Processing KLGYO.IS...
Processing TNZTP.IS...
Processing CMBTN.IS...
Processing AYEN.IS...
Processing ATAKP.IS...
Processing ESEN.IS...
Processing GEDIK.IS...
Processing KRVGD.IS...
Processing MNDTR.IS...
Processing GSRAY.IS...
Processing ISFIN.IS...
Processing RALYH.IS...
Processing GLCVY.IS...
Processing KZBGY.IS...
Processing PNLSN.IS...
Processing TRCAS.IS...
Processing BOSSA.IS...
Processing ALGYO.IS...
Processing KONKA.IS...
Processing ALKA.IS...
Processing INTEM.IS...
Processing MAALT.IS...
Processing SAYAS.IS...
Processing BRLSM.IS...
Processing GOODY.IS...
Processing SONM

In [29]:
divergences_df

,Ticker,Type,First Date,Second Date,First Price,Second Price,First Close,Second Close,First RSI,Second RSI,Final Close,Final Date,Next Bar Close
0,QNBFB.IS,Bearish,2023-09-15 00:00:00+03:00,2023-09-19 00:00:00+03:00,202.800003,200.800003,202.800003,200.800003,81.865979,70.033613,287.750000,2023-12-29 00:00:00+03:00,220.800003
1,QNBFB.IS,Bearish,2023-09-29 00:00:00+03:00,2023-10-05 00:00:00+03:00,403.899994,324.000000,403.899994,324.000000,89.902639,60.538621,287.750000,2023-12-29 00:00:00+03:00,337.000000
2,QNBFB.IS,Bearish,2023-10-06 00:00:00+03:00,2023-10-10 00:00:00+03:00,337.000000,333.600006,337.000000,333.600006,64.310289,58.726868,287.750000,2023-12-29 00:00:00+03:00,366.899994
3,QNBFB.IS,Bearish,2023-10-11 00:00:00+03:00,2023-10-16 00:00:00+03:00,366.899994,346.799988,366.899994,346.799988,62.646142,56.534596,287.750000,2023-12-29 00:00:00+03:00,346.799988
4,QNBFB.IS,Bearish,2023-10-16 00:00:00+03:00,2023-10-19 00:00:00+03:00,346.799988,343.399994,346.799988,343.399994,56.534596,54.177929,287.750000,2023-12-29 00:00:00+03:00,366.399994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8079,SANFM.IS,Bullish,2023-10-30 00:00:00+03:00,2023-11-07 00:00:00+03:00,30.000000,36.639999,30.000000,36.639999,68.050868,79.278339,39.119999,2023-12-29 00:00:00+03:00,36.400002
8080,SANFM.IS,Bullish,2023-11-09 00:00:00+03:00,2023-11-17 00:00:00+03:00,35.580002,43.000000,35.580002,43.000000,75.904030,83.600610,39.119999,2023-12-29 00:00:00+03:00,41.080002
8081,SANFM.IS,Bullish,2023-11-21 00:00:00+03:00,2023-11-27 00:00:00+03:00,39.099998,48.520000,39.099998,48.520000,70.037833,76.607827,39.119999,2023-12-29 00:00:00+03:00,47.560001
8082,SANFM.IS,Bullish,2023-11-30 00:00:00+03:00,2023-12-06 00:00:00+03:00,46.599998,51.000000,46.599998,51.000000,72.118795,76.254074,39.119999,2023-12-29 00:00:00+03:00,50.000000


In [30]:
# Save to CSV
divergences_df.to_csv('test-detected_divergences_for_multiple_stocks_with_next_bar_close.csv', index=False)